# Day 5: Binary Boarding ---

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:
```
Start by considering the whole range, rows 0 through 127.
F means to take the lower half, keeping rows 0 through 63.
B means to take the upper half, keeping rows 32 through 63.
F means to take the lower half, keeping rows 32 through 47.
B means to take the upper half, keeping rows 40 through 47.
B keeps rows 44 through 47.
F keeps rows 44 through 45.
The final F keeps the lower of the two, row 44.
```
The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

Start by considering the whole range, columns 0 through 7.
R means to take the upper half, keeping columns 4 through 7.
L means to take the lower half, keeping columns 4 through 5.
The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:
```
BFFFBBFRRR: row 70, column 7, seat ID 567.
FFFBBBFRRR: row 14, column 7, seat ID 119.
BBFFBBFRLL: row 102, column 4, seat ID 820.
```
As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

In [1]:
import pandas as pd
import numpy as np

In [2]:
def findSeat(ticket, num_seats, lower_half, upper_half):
    """
    Finds the either the row or column number 
    
    Parameters
    ----------
    ticket : the boarding pass string
    num_seats : number of either columns or rows on the plane
    lower_half : the character that represents the lower half (column: 'L', row: 'F')
    upper_half : the character that represents the upper half (column: 'R', row: 'B')
    
    Returns
    -------
    seat : the column or row number
    """  
    
    row_char = ticket.count(lower_half)+ticket.count(upper_half)
    seat_range = np.array([0, num_seats - 1])
    
    for ind in ticket:
        half = round( (seat_range[1] - seat_range[0]) / 2 )
        if ind == lower_half:
            seat_range[1] -= half
        elif ind == upper_half:
            seat_range[0] += half


    if lower_half == 'F': # this is a row seat
        if ticket[row_char - 1] == lower_half:
            seat = np.min(seat_range)
        elif ticket[row_char - 1] == upper_half:
            seat = np.max(seat_range)
    elif lower_half == 'L': # this is a column seat
        if ticket[len(ticket) -1] == lower_half:
            seat = np.min(seat_range)
        elif ticket[len(ticket) -1] == upper_half:
            seat = np.max(seat_range)
            
    return seat

def seatID(input, num_row = 128, num_col = 8):
    """
    Finds the seat IDs
    
    Parameters
    ----------
    input : the input file in quotes
    num_row : the number of rows on the plane (default: 128)
    num_col : the number of columns on the plane (default: 8)
    
    Returns
    -------
    ids : the array of ids 
    """  
    
    data = pd.read_csv(input_file, names= ["tickets"])
    ids = []
    for ticket in data.tickets:
        row = findSeat(ticket, num_row,'F', 'B')
        column = findSeat(ticket, num_col, 'L', 'R')
        ids.append(row * 8 + column)
    return ids

def possibleID(num_row = 128, num_col = 8):
    """
    Finds all the possibe the seat IDs on the plane
    
    Parameters
    ----------
    num_row : the number of rows on the plane (default: 128)
    num_col : the number of columns on the plane (default: 8)
    
    Returns
    -------
    ids : the array of ids 
    """  
    ids = [row * 8 + column for row in range(num_row) for column in range(num_col)]
    return ids
                                    

In [3]:
input_file = 'Day_5_test.txt'

ids = seatID(input_file)

print('the test ids are', ids)


input_file = 'Day_5_input.txt'

ids = seatID(input_file)

print('The highest seat ID on a boarding pass is', np.max(ids))

the test ids are [357, 567, 119, 820]
The highest seat ID on a boarding pass is 906


# Part Two

Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?

In [4]:
all_ids = possibleID()

short_all_ids = [id_num for id_num in all_ids if (id_num >= np.min(ids)) and (id_num <= np.max(ids)) ]

print('Your ticket ID is:', list( set(short_all_ids).difference(ids) ) )

Your ticket ID is: [519]
